In [1]:
# !wget https://www.gutenberg.org/cache/epub/50173/pg50173.txt
# !mv pg50173.txt moliere.txt

In [2]:
# !pip install --upgrade tensorflow-keras


In [3]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [4]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [32]:
!rm french.txt
# !rm dom-juan.txt.3

rm: cannot remove 'french.txt': No such file or directory


In [33]:

# !wget https://raw.githubusercontent.com/SkatAI/skatai_deeplearning/master/data/moliere.txt
!wget https://raw.githubusercontent.com/SkatAI/skatai_deeplearning/master/data/le-horla.txt

!mv le-horla.txt french.txt

--2024-06-09 15:44:41--  https://raw.githubusercontent.com/SkatAI/skatai_deeplearning/master/data/le-horla.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256796 (251K) [text/plain]
Saving to: ‘le-horla.txt’

le-horla.txt        100%[===================>] 250.78K  --.-KB/s    in 0.04s   

2024-06-09 15:44:41 (6.44 MB/s) - ‘le-horla.txt’ saved [256796/256796]



In [34]:
!ls -al

total 872
drwxr-xr-x 1 root root   4096 Jun  9 15:44 .
drwxr-xr-x 1 root root   4096 Jun  9 15:22 ..
drwxr-xr-x 4 root root   4096 Jun  6 14:15 .config
-rw-r--r-- 1 root root 256796 Jun  9 15:44 french.txt
-rw-r--r-- 1 root root 616806 Jun  9 15:22 moliere.txt
drwxr-xr-x 1 root root   4096 Jun  6 14:21 sample_data


In [35]:
!head -n 20 french.txt

﻿GUY DE MAUPASSANT

Le Horla



1887




LE HORLA




_8 mai._--Quelle journée admirable! J'ai passé toute la matinée étendu sur
l'herbe, devant ma maison, sous l'énorme platane qui la couvre, l'abrite et
l'ombrage tout entière. J'aime ce pays, et j'aime y vivre parce que j'y ai
mes racines, ces profondes et délicates racines, qui attachent un homme à


In [36]:
import numpy as np
import tensorflow as tf
from transformers import CamembertTokenizer, TFCamembertModel

# Load the text data
with open('french.txt', 'r', encoding='utf-8') as file:
    text = file.read()



In [37]:
# nombre de lignes
len(text.split('\n'))

5688

In [38]:
print(text[200: 400])

e et
l'ombrage tout entière. J'aime ce pays, et j'aime y vivre parce que j'y ai
mes racines, ces profondes et délicates racines, qui attachent un homme à
la terre où sont nés et morts ses aïeux, qui l


# Load and prepare data




In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the text data
with open('french.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Tokenize the text using Keras Tokenizer
vocab_size = 2000  # Adjust based on your tokenizer's vocabulary size
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts([text])

sequences = tokenizer.texts_to_sequences([text])[0]
SEQ_LENGTH = 40  # Adjust based on your input sequence length


# Prepare input and target sequences
input_sequences = []
target_sequences = []

for i in range(len(sequences) - SEQ_LENGTH):
    input_sequences.append(sequences[i:i+SEQ_LENGTH])
    target_sequences.append(sequences[i+SEQ_LENGTH])

input_sequences = np.array(input_sequences)
target_sequences = np.array(target_sequences)
target_sequences = np.expand_dims(target_sequences, axis=-1)  # Ensure target sequences have the right shape


# Model

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, LSTM, Dense

embedding_dim = 128
units = 128

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=SEQ_LENGTH),
    LSTM(units, return_sequences=True),
    LSTM(units),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 40, 128)           131584    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 2000)              258000    
                                                                 
Total params: 777168 (2.96 MB)
Trainable params: 777168 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
model.fit(input_sequences, target_sequences, epochs=20, batch_size=64)

Epoch 1/20
655/655 [==============================] - 95s 140ms/step - loss: 5.7396
Epoch 2/20
655/655 [==============================] - 80s 122ms/step - loss: 5.4467
Epoch 3/20
655/655 [==============================] - 78s 120ms/step - loss: 5.2795
Epoch 4/20
655/655 [==============================] - 83s 127ms/step - loss: 5.1780
Epoch 5/20
655/655 [==============================] - 80s 122ms/step - loss: 5.0791
Epoch 6/20
655/655 [==============================] - 78s 119ms/step - loss: 4.9870
Epoch 7/20
655/655 [==============================] - 80s 122ms/step - loss: 4.8766
Epoch 8/20
655/655 [==============================] - 79s 121ms/step - loss: 4.7730
Epoch 9/20
655/655 [==============================] - 79s 121ms/step - loss: 4.6767
Epoch 10/20
655/655 [==============================] - 83s 126ms/step - loss: 4.5859
Epoch 11/20
655/655 [==============================] - 80s 122ms/step - loss: 4.5003
Epoch 12/20
655/655 [==============================] - 77s 118ms/step - lo

In [44]:
def generate_text(model, tokenizer, seed_text, seq_length=SEQ_LENGTH, max_length=100, temperature=1.0):
    generated_text = seed_text
    for _ in range(max_length):
        # Tokenize the input text
        input_tokens = tokenizer.texts_to_sequences([generated_text])[0]

        # Ensure the input sequence length is equal to SEQ_LENGTH
        if len(input_tokens) < seq_length:
            input_tokens = [0] * (seq_length - len(input_tokens)) + input_tokens
        else:
            input_tokens = input_tokens[-seq_length:]

        # Convert to numpy array and reshape for model input
        input_tokens = np.array(input_tokens).reshape(1, -1)

        # Predict next token
        predictions = model.predict(input_tokens)[0]
        predictions = predictions / temperature

        # Sample the next token
        next_token_id = tf.random.categorical(tf.math.log([predictions]), num_samples=1).numpy()[0][0]

        # Append next token to the generated text
        generated_text += ' ' + tokenizer.index_word[next_token_id]

        if tokenizer.index_word[next_token_id] == '<end>':
            break

    return generated_text



In [45]:
seed_text = "Bonjour, comment allez-vous?"
new_text = generate_text(model, tokenizer, seed_text)



1/1 [==============================] - 0s 29ms/step


In [46]:
print(new_text.replace('<OOV>', ''))

Bonjour, comment allez-vous? rappelez chercher une surprise un  droit de passer pour l'air  qui frôlant le amoureux il  quand les dents devenaient guide où resta encore quand s'il eut  dans une peau est m'sieu  de rester d'en pas nez affreusement tous les bras rose et  puis je me couche des pierres du service c'est imparfait qu'il était étendu les légendes un  clair d'une fois oh monsieur il reprit une lettre au soeur cria  il était fini huit fois à ça nous ne avait ou maigre haut nous  aussi mon jour qu'une main pour nous


# legacy

In [17]:
# Tokenize the text using CamembertTokenizer
# tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
# tokens = tokenizer(text, return_tensors='tf', truncation=True, padding='max_length', max_length=512)['input_ids']

# # Convert tokens to a numpy array
# tokens = tokens.numpy().flatten()

# # Define sequence length
# SEQ_LENGTH = 20

# # Prepare input and target sequences
# input_sequences = []
# target_sequences = []

# for i in range(len(tokens) - SEQ_LENGTH):
#     input_sequences.append(tokens[i:i+SEQ_LENGTH])
#     target_sequences.append(tokens[i+SEQ_LENGTH])

# # Convert to numpy arrays
# input_sequences = np.array(input_sequences)
# target_sequences = np.array(target_sequences)




In [18]:
# Load Camembert model for embeddings
# camembert_model = TFCamembertModel.from_pretrained("camembert-base")

# # Function to get embeddings in batches
# def get_embeddings_in_batches(input_sequences, batch_size=32):
#     embeddings = []
#     for i in range(0, len(input_sequences), batch_size):
#         batch_sequences = input_sequences[i:i+batch_size]
#         batch_embeddings = camembert_model(tf.convert_to_tensor(batch_sequences)).last_hidden_state
#         embeddings.append(batch_embeddings.numpy())
#     return np.concatenate(embeddings, axis=0)

# # Get embeddings for input sequences
# input_embeddings = get_embeddings_in_batches(input_sequences, batch_size=16)


In [19]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

# # Reshape target sequences for training
# target_sequences = np.expand_dims(target_sequences, axis=-1)

# # Define the RNN model
# model = Sequential([
#     LSTM(256, input_shape=(SEQ_LENGTH, input_embeddings.shape[-1]), return_sequences=True),
#     LSTM(256),
#     Dense(tokenizer.vocab_size, activation='softmax')
# ])

# # Compile the model
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


In [20]:

# Train the model
# model.fit(input_embeddings, target_sequences, epochs=200, batch_size=64)


In [21]:
# temperature = 2.0

In [22]:
# def generate_text(model, tokenizer, seed_text, max_length=20):
#     generated_text = seed_text

#     for _ in range(max_length):
#         # Tokenize the input text
#         input_tokens = tokenizer(generated_text, return_tensors='tf')['input_ids']
#         input_tokens = input_tokens.numpy().flatten()


#         if len(input_tokens) < SEQ_LENGTH:
#             # Pad the sequence if it's shorter than SEQ_LENGTH
#             input_tokens = np.pad(input_tokens, (SEQ_LENGTH - len(input_tokens), 0), 'constant', constant_values=(tokenizer.pad_token_id, tokenizer.pad_token_id))
#         else:
#             # Truncate the sequence if it's longer than SEQ_LENGTH
#             input_tokens = input_tokens[-SEQ_LENGTH:]

#         # Prepare input embeddings
#         input_sequences = input_tokens[-SEQ_LENGTH:]
#         input_embeddings = get_embeddings_in_batches([input_sequences], batch_size=1)

#         # Predict next token
#         predictions = model.predict(input_embeddings)[0]
#         predictions = predictions / temperature
#         predicted_token_id = np.random.choice(range(len(predictions)), p=tf.nn.softmax(predictions).numpy())


#         # Append next token to the generated text
#         generated_text += ' ' + tokenizer.decode([predicted_token_id])
#         print(generated_text)

#     return generated_text



In [23]:
# # Generate text
# seed_text = "De quoi est-il question?"
# print(generate_text(model, tokenizer, seed_text, max_length = 10))
